# Object validation for obeservation

In [2]:
import numpy as np
import pandas as pd
import re
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

ms=pd.read_csv('messier_objects.csv', low_memory=False)
ident = np.array(ms['ID (for resolver)'])
name = np.array(ms['Common Name'])

## Search object

In [3]:
while True:
    inp = input('Name of Object -- ',)
    inp2 = inp.split()


    if all([x.isalpha() for x in inp2]):
        inp = inp.title()
        find_res = np.array([x.find(inp) for x in name])
        pos = np.where(find_res!=-1)[0]
        if len(pos)==0:
            print('No such object. Try again.')
            continue
        ra, dec = ms['RAJ2000'][pos].values, ms['DEJ2000'][pos].values
        if len(name[pos])>1:
            print('Multiple matches found. Try again.')
            continue
        else:
            print(name[pos][0],ra[0],dec[0])
            obj=ident[pos][0]
        break
    
    elif all([x.isalnum() for x in inp2]): 
        inp = inp.title()
        inp = re.split('(\d+)',inp)
        inp[0] = inp[0].rstrip()
        inp = inp[:2]
        inp = ' '.join(inp)
        find_res = np.array([x.find(inp) for x in ident])
        pos = np.where(find_res!=-1)[0]
        if len(pos)==0:
            print('No such object. Try again.')
            continue
        ra, dec = ms['RAJ2000'][pos].values, ms['DEJ2000'][pos].values
        print(ident[pos][0],ra[0],dec[0])
        obj=ident[pos][0]
        break
        
    else:
        print('No such object. Try again.')
# obj

Name of Object --  m1


M 1 83.63308 22.0145


## Enter location on earth

In [4]:
while True:
    location = input("Enter latitude and longitude:")
    # format in decimal degrees with - for south and west; 
    # example: -42.75 +24.5 indicates 42.75 degrees S and 24.5 degrees E
    lis = location.split()
    if not len(lis) == 2:
        print("Enter location again")
        continue
    else:
        try:
            x = float(lis[0])
            y = float(lis[1])
            if -90<x<90 and -180<y<180:
                lati = x
                long = y
                print(x,' deg lat, ',y,' deg lon')
                break
            else:
                print("Enter location again")
                continue
        except:
            print("Enter location again")
            continue
# variables are 
# lati
# long

Enter latitude and longitude: 10 70


10.0  deg lat,  70.0  deg lon


## Enter time of observation

In [5]:
while True:
    timexx = input("Enter time as hhmm:")
    # time in IST 
    timexx = str(timexx)
    if len(timexx) == 4:
        try:
            h = int(timexx[:2])
            m = int(timexx[-2:])
            if 0<=h<=23 and 0<=m<=59:
                hour = h
                minute = m
                print(hour,'h ',minute,'m')
                break
        except:
            print("Enter time again")
            continue
    else:
        print("Enter time again")
        continue
# variables are
# hour
# minute

Enter time as hhmm: 2200


22 h  0 m


## Enter date

In [6]:
while True:
    datexx = input("Enter data as yyyy-mm-dd:")
#     datexx = datexx.split('-')
    if re.match("([12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]))", datexx):
        datex = datexx
        print
        (datex)
        break
    else:
        print("Enter date again")
        continue
# variable is datex

Enter data as yyyy-mm-dd: 2020-02-02


## Calculate Alt-Azimuth

In [7]:
loc = EarthLocation(lat=lati*u.deg, lon=long*u.deg) 
utcoffset = 5.5*u.hour  # IST
time = Time('{} {}:{}:00'.format(datex, hour, minute)) - utcoffset
obj_coord = SkyCoord.from_name(obj)
objaltaz = obj_coord.transform_to(AltAz(obstime=time,location=loc))

print("Altitude/Azimuth = {0.alt:.5} / {1.az:.5}\nDate = {2}\nTime = {3}:{4}\nLocation = {5} deg lat, {6} deg lon \nObject: {7}".format(objaltaz,objaltaz,datex,hour,minute,lati,long,obj))

Altitude/Azimuth = 76.709 deg / 335.51 deg
Date = 2020-02-02
Time = 22:0
Location = 10.0 deg lat, 70.0 deg lon 
Object: M 1


In [8]:
obj_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63308333, 22.0145)>

In [46]:
al='{0.alt:.5}'.format(objaltaz)
az='{0.az:.5}'.format(objaltaz)
al=float(al.split(' ')[0])
az=float(az.split(' ')[0])

if al>0: # altitude is positive
    print("{} in sky".format(obj))
else: # altitude is negative
    print("{} not in sky".format(obj))

M 1 not in sky
